# Classification with Reduced Dimensionality

## Handwritten Digits - Reducing with PCA

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import datasets

In [2]:
digits = datasets.load_digits()

features = StandardScaler().fit_transform(digits.data)

pca = PCA(n_components=0.99, whiten=True)

features_pca = pca.fit_transform(features)

print(features.shape[1])
print(features_pca.shape[1])

64
54


## Small Wiki Corpus - Reducing with TruncatedSVD

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

import numpy as np
import pandas as pd

In [5]:
corpus = pd.read_csv("corpora/german_wiki_small.csv")
corpus.head(2)

,category,length,text,textlength
0,Computerspiel nach Plattform,184,"SWIV ist ein Shoot ' em up von Storm , erschie...",184
1,Computerspiel nach Plattform,342,Slayer ist ein Action-Rollenspiel des Entwickl...,342


In [8]:
labels = corpus["category"]
tfidf  = TfidfVectorizer().fit_transform(corpus["text"])

svd = TruncatedSVD(n_components=10, n_iter=10, random_state=42) 
vector = svd.fit_transform(tfidf) 

X_train, X_test, y_train, y_test = train_test_split(vector, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    train_size=0.8,
                                                    random_state=42)

### Standard approach

In [11]:
clf = LogisticRegression(solver="lbfgs", multi_class="ovr")
model = clf.fit(X_train, y_train)

scores = cross_val_score(clf, vector, labels, cv=5)
mean = np.mean(scores)

print("Der Mittelwert der cross validation bei der Klassifizierung " 
      + f"ist {str(np.around(mean, decimals=3))}." + "\n")


y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average="micro")

print("Der F1-score für die Klassifizierung ist " + f"{str(np.around(f1, decimals=3))}.")

Der Mittelwert der cross validation bei der Klassifizierung ist 0.774.

Der F1-score für die Klassifizierung ist 0.83.


### Pipeline approach

In [51]:
pipe = Pipeline([('svd', TruncatedSVD(n_components=9, n_iter=10)),
                 ('clf', LogisticRegression())])

pipe.fit(X_train, y_train)

scores = cross_val_score(pipe, vector, labels, cv=5)
mean = np.mean(scores)

print("Der Mittelwert der cross validation bei der Klassifizierung " 
      + f"ist {str(np.around(mean, decimals=3))}." + "\n")


y_pred = pipe.predict(X_test)
f1 = f1_score(y_test, y_pred, average="micro")

print("Der F1-score für die Klassifizierung ist " + f"{str(np.around(f1, decimals=3))}.")

Der Mittelwert der cross validation bei der Klassifizierung ist 0.755.

Der F1-score für die Klassifizierung ist 0.784.
